## Objectives:  
    
We want to set up the data preprocessing step in the single-cell scenario, in which way the pretraining will be  conducted via lising all the genes and their corresponding regulation motifs.

In this notebook, we will cover the following contents:

- Visualization of the number of gene and mapped motifs to make sure the input length is handlable.


In [2]:
from pathlib import Path

import numpy as np
import snapatac2 as snap
from gcell._settings import get_setting
import pandas as pd
from pyranges import PyRanges as pr
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
save_dir = "/scratch/project_465001820/scCLIP/dataset/processed_pbmc_dataset"

Prerequisites: download the following files from 10X

```bash
wget "https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_granulocyte_sorted_10k/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5"
```

Loading the single-cell multiomics data

In [4]:
ad = sc.read_10x_h5('/scratch/project_465001820/scCLIP/dataset/pbmc_multiomics/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5', gex_only=False)
ad

/scratch/project_465001820/miniconda3/envs/scCLIP2/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 11898 × 180488
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

## Getting the cell type labels

In [5]:
# %%
# read rna data
if not Path(f'{save_dir}/rna.h5ad').exists():
    rna = snap.read(snap.datasets.pbmc10k_multiome(modality='RNA'), backed=None)
    sc.pp.highly_variable_genes(rna, flavor='seurat_v3', n_top_genes=3000)
    rna_filtered = rna[:, rna.var.highly_variable]
    sc.pp.normalize_total(rna_filtered, target_sum=1e4)
    sc.pp.log1p(rna_filtered)
    snap.tl.spectral(rna_filtered, features=None)
    snap.tl.umap(rna_filtered)
    rna_filtered.write(f'{save_dir}/rna.h5ad')
else:
    rna_filtered = sc.read(f'{save_dir}/rna.h5ad')

Here, I'm going to use the cell type labels from preprocessed snapatac2 object.

In [6]:
ad = ad[ad.obs.index.isin(rna_filtered.obs.index.values)]

In [7]:
rna_filtered

AnnData object with n_obs × n_vars = 9631 × 3000
    obs: 'domain', 'cell_type'
    var: 'gene_ids', 'feature_types', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'log1p', 'spectral_eigenvalue'
    obsm: 'X_spectral', 'X_umap'

In [8]:
barcode_to_celltype = rna_filtered.obs.to_dict()['cell_type']
ad.obs['cell_type'] = ad.obs.index.map(barcode_to_celltype)


/tmp/ipykernel_64299/1270784820.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['cell_type'] = ad.obs.index.map(barcode_to_celltype)
/scratch/project_465001820/miniconda3/envs/scCLIP2/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/scratch/project_465001820/miniconda3/envs/scCLIP2/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [9]:
ad

AnnData object with n_obs × n_vars = 9627 × 180488
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

In [10]:
ad.obs

,cell_type
AAACAGCCAATCCCTT-1,CD4 TCM
AAACAGCCAATGCGCT-1,CD4 Naive
AAACAGCCACCAACCG-1,CD8 Naive
AAACAGCCAGGATAAC-1,CD4 Naive
AAACAGCCAGTTTACG-1,CD4 TCM
...,...
TTTGTTGGTGACATGC-1,CD8 Naive
TTTGTTGGTGTTAAAC-1,CD8 Naive
TTTGTTGGTTAGGATT-1,NK
TTTGTTGGTTGGTTAG-1,CD4 TCM


# Extracting the ATAC-seq dataset 

In [17]:
import pandas as pd
from pyranges import PyRanges as pr
def get_peak_from_snapatac(atac: snap.AnnData):
    """
    Get the peak names from the snapatac object.

    Args:
        atac: snapatac2 processed AnnData object

    Returns:
        peak_names: pandas DatasFrame with the peak names
    """
    peak_names = pd.DataFrame(atac.var.index.str.split('[:-]').tolist(), columns=['Chromosome', 'Start', 'End'])
    peak_names['Start'] = peak_names['Start'].astype(int)
    peak_names['End'] = peak_names['End'].astype(int)
    return peak_names

peaks = get_peak_from_snapatac(ad_atac)
peaks.shape

(143887, 3)

In [11]:
ad_atac = ad[:, np.where(ad.var.feature_types == 'Peaks')[0]]
ad_atac

View of AnnData object with n_obs × n_vars = 9627 × 143887
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

We filter for cell types with >100 cells and at least 3M library size.

In [12]:
# %%
cell_number = ad_atac.obs.groupby('cell_type', observed=False).size().to_dict()
print("The following cell types have more than 100 cells and library size > 3M, adding them to celltype_for_modeling")
celltype_for_modeling = []
for cell_type in cell_number:
    if cell_number[cell_type] > 100:
        celltype_for_modeling.append(cell_type)
        libsize = int(ad_atac[ad_atac.obs.cell_type == cell_type].X.sum())
        if libsize > 3000000:
            print(f"{cell_type} number of cells: {cell_number[cell_type]}, library size: {libsize}")

The following cell types have more than 100 cells and library size > 3M, adding them to celltype_for_modeling
CD14 Mono number of cells: 2551, library size: 62039712
CD16 Mono number of cells: 442, library size: 10504281
CD4 Naive number of cells: 1382, library size: 29601700
CD4 TCM number of cells: 1113, library size: 24745884
CD4 TEM number of cells: 286, library size: 6965094
CD8 Naive number of cells: 1353, library size: 30763516
CD8 TEM_1 number of cells: 322, library size: 7209424
CD8 TEM_2 number of cells: 315, library size: 6378479
Intermediate B number of cells: 300, library size: 8366581
MAIT number of cells: 130, library size: 3290943
Memory B number of cells: 298, library size: 7267452
NK number of cells: 403, library size: 8665433
Naive B number of cells: 125, library size: 3618917
Treg number of cells: 157, library size: 3135736
cDC number of cells: 180, library size: 6102032
gdT number of cells: 143, library size: 3106695


In [18]:
# %%
def get_peak_acpm_for_cell_type(atac: snap.AnnData, cell_type: str):
    """
    Get the peak acpm for a given cell type.
    """
    peaks = get_peak_from_snapatac(atac) #the coordinates of all the peaks from 10x pbmc atac dataset
    counts = np.array(atac[atac.obs.cell_type == cell_type].X.sum(0)).flatten()#counts of each peak
    acpm = np.log10(counts / counts.sum() * 1e5 + 1) #normalization
    peaks['aCPM'] = acpm/acpm.max() #to 0-1, and this is a pseudobulk result
    peaks = peaks.query('Chromosome.str.startswith("chr") & ~Chromosome.str.endswith("M") & ~Chromosome.str.endswith("Y") & ~Chromosome.str.startswith("chrUn")')
    peaks = pr(peaks, int64=True).sort().df
    return peaks

save the peak ".bed" files for each cell type 

In [16]:
for cell_type in celltype_for_modeling:
    peaks = get_peak_acpm_for_cell_type(ad_atac, cell_type)
    peaks.to_csv(f'{save_dir}/{cell_type.replace(" ", "_").lower()}.atac.bed', sep='\t', index=False, header=False)

NameError: name 'get_peak_from_snapatac' is not defined

Plot the library size for each cell

In [ ]:
libsize = ad_atac.X.sum(axis=1)

In [ ]:
#to a binary representation
plt.figure(figsize=(10, 6))
sns.kdeplot(libsize, fill=True)
plt.xlabel("Library Size", fontsize = 12)
plt.ylabel("Density", fontsize = 12)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.title("Density Plot of the ATAC-seq library size")
plt.savefig("../figures/pbmc_libsize_density_plot.png", dpi=300)
plt.show()

# Query motifs 

In [ ]:
import os
from pathlib import Path

from gcell._settings import get_setting
import sys
sys.path.append("/scratch/project_465001820/scCLIP/get_model/tutorials/")
from preprocess_utils import (
    add_atpm,
    add_exp,
    create_peak_motif,
    download_motif,
    get_motif,
    query_motif,
)

query motifs according to the chromosome

In [ ]:
motif_bed = "/scratch/project_465001820/scCLIP/dataset/motif/hg38.archetype_motifs.v1.0.bed.gz"
peak_bed = f"{save_dir}/cd4_naive.atac.bed" # since all cell types share the same peak set, when querying motifs, we can just use one cell type to query motifs.
peaks_motif = query_motif(peak_bed, motif_bed, save_dir)#filtering the motifs that are overlap with the peak 
get_motif_output = get_motif(peak_bed, peaks_motif, save_dir)#overlapping the motif according to the chromosomes
get_motif_output = f"{save_dir}/get_motif.bed"

# Create peak "motif zarr file

Create a peak x motif matrix stored in a zarr file. If you are working on multiple cell types with the same peak set, you can use the same peak bed and zarr file for all cell types.

In [ ]:
create_peak_motif(get_motif_output, f"{save_dir}/pbmc10k_multiome.zarr", peak_bed) # all cell types will later be added to the same zarr file as we use the same peak set.


Add aCPM data to region x motif matrix

In [ ]:
for cell_type in celltype_for_modeling:
    print(f"{cell_type}")
    add_atpm(
        f"{save_dir}/pbmc10k_multiome.zarr",
        f"{save_dir}/{cell_type}.atac.bed",
        cell_type,
    )

Add expression and TSS data to region x motif matrix

In [ ]:
for cell_type in celltype_for_modeling:
    add_exp(
        f"{save_dir}/pbmc10k_multiome.zarr",
        f"{save_dir}/{cell_type}.rna.csv",
        f"{save_dir}/{cell_type}.atac.bed",
        cell_type,
        assembly="hg38",
        version=44,
        extend_bp=300, # extend TSS region to 300bp upstream and downstream when overlapping with peaks
    id_or_name="gene_name", # use gene_name or gene_id to match the gene expression data, checkout your rna.csv file column names, should be either [gene_name, TPM] or [gene_id, TPM]
)